In [1]:
import numpy as np
from pathlib import Path
import yaml
import random


In [2]:
dataset = Path("/home/konstanty/STUDIA/masters_1_sem/rob3/data_odometry_velodyne")
predictions = Path("/home/konstanty/STUDIA/masters_1_sem/rob3/semantic-segmentation/predictions/")
with open("semantic-kitti.yaml", "r") as f:
    kitty_conf = yaml.load(f, yaml.Loader)

In [3]:
def load_labels(file_path):
    labels = np.fromfile(file_path, dtype=np.uint32)
    semantic_labels = labels & 0xFFFF  # mask lower 16 bits
    instance_ids = labels >> 16  # upper 16 bits
    return semantic_labels, instance_ids
def translate_element(x):
    translation_key =kitty_conf["learning_map"]
    return translation_key.get(x, x) 
    

In [4]:
files = frame_files = list((predictions / "sequences/08/predictions").iterdir())

In [5]:
sampled_files = random.sample(files, int(len(files) * 0.01))

In [6]:
l = []
for f in sampled_files:
    label_file = (
        dataset
        / "dataset"
        / "sequences"
        / f.parents[1].stem
        / "labels"
        / (f.stem + ".label")
    )
    a, _ = load_labels(label_file)

    l.append(np.vectorize(translate_element)(a))

In [7]:
L = np.concatenate(l, axis=0) 

In [8]:
matrices = [np.load(f) for f in sampled_files]
M = np.concatenate(matrices, axis=0) 

In [9]:

def fitness(w):
  # Ensure w is a numpy array for operations like M*w
  w_array = np.array(w)
  if M.shape[1] != w_array.shape[0]:
      raise ValueError("M and w dimensions are incompatible for multiplication")
  return (np.mean(np.argmax((M*w_array),axis=1) == L),)

In [ ]:
def fitness_mIoU(w):
    # Ensure w is a numpy array for vectorized operations
    w_array = np.array(w)

    # 1. Apply weights and get predictions
    # Assuming M is (num_samples, num_classes) and w_array is (num_classes,)
    # If M is (num_samples, num_features) and w is (num_features,),
    # you might need to reshape M or w, or apply a matrix multiplication.
    # The original `M*w` suggests element-wise multiplication or broadcasting.
    # Let's assume `M` contains the pre-weighted scores if `w` is applied earlier,
    # or `w` scales the class-specific scores in `M`.
    # A common interpretation for `M*w` where `w` is weights for classes:
    # If M is (num_samples, num_classes), and w is (num_classes,), then M * w
    # would broadcast w across rows, scaling each class's score.
    # Example: If M[i,j] is the score for sample i, class j, and w[j] is the weight for class j.
    weighted_scores = M * w_array

    # Get the predicted class for each sample/pixel
    predictions = np.argmax(weighted_scores, axis=1)

    unique_classes = kitty_conf["learning_map"]
    iou_per_class = []

    for cls in unique_classes:
        # True Positives: Pixels correctly predicted as class `cls`
        tp = np.sum((predictions == cls) & (L == cls))

        # False Positives: Pixels predicted as class `cls` but are actually other classes
        fp = np.sum((predictions == cls) & (L != cls))

        # False Negatives: Pixels that are class `cls` but predicted as other classes
        fn = np.sum((predictions != cls) & (L == cls))

        # Calculate IoU for the current class
        denominator = tp + fp + fn
        if denominator == 0:
            iou = 1.0  # If there are no true positives, false positives, or false negatives for this class,
        # and it's not present in ground truth or predictions, IoU is 1.0.
        # This case needs careful consideration. If the class is not present in L at all,
        # it's often ignored. If it's present but not predicted/true, then 0.
        # For simplicity, if the denominator is 0, it means the class was
        # neither present in ground truth nor predicted, so IoU is 1.
        # However, a common practice is to only calculate for classes present in ground truth.
        else:
            iou = tp / denominator
        iou_per_class.append(iou)

    if not iou_per_class:
        return (0.0,)  # No classes found, mIoU is 0

    mIoU = np.mean(iou_per_class)
    return (mIoU,)  # DEAP fitness functions must return a tuple


In [11]:
import torch

# M_tensor = torch.tensor(M, dtype=torch.float32, device="cuda")  # shape: (N, 19)
# y_tensor = torch.tensor(L, dtype=torch.long, device="cuda")  # shape: (N,)

# def evaluate(individual):
#     w = torch.tensor(individual, dtype=torch.float32, device="cuda")  # shape: (19,)
    
#     logits = M_tensor * w  # shape: (N, 19), broadcasted
    
#     predictions = torch.argmax(logits, dim=1)  # shape: (N,)
    
#     accuracy = (predictions == y_tensor).float().mean().item()
    
#     return (accuracy,)

In [12]:
for _ in range(5):
	w = np.random.random(19)
	print(fitness_mIoU(w))

(np.float64(0.7829675333335407),)
(np.float64(0.8068435580726168),)
(np.float64(0.7909755121285638),)
(np.float64(0.8068141635887778),)
(np.float64(0.8065411965294897),)


In [13]:
from deap import base, creator, tools, algorithms

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # maximize accuracy
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", lambda: np.random.uniform(0.01, 50.0))  # avoid 0
toolbox.register(
    "individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=19
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness_mIoU)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("select", tools.selTournament, tournsize=3)

# Optional: enforce positivity after mutation
def repair(individual):
    # This function should ensure the output is an Individual object
    repaired_list = list(individual) # Ensure it's a list for modification
    for i in range(len(repaired_list)):
        if repaired_list[i] <= 0:
            repaired_list[i] = np.random.uniform(0.01, 1.0)
    return creator.Individual(repaired_list) # Cast back to Individual

# Decorating to enforce constraints
def safe_mutate(individual):
    mutant, = tools.mutGaussian(individual, mu=1.0, sigma=0.5, indpb=0.2)
    # Ensure that `repaired` is an instance of creator.Individual
    repaired = repair(mutant) # repair now returns an Individual
    return repaired,  # NOTE: this is a TUPLE!

toolbox.register("mutate", safe_mutate)

population = toolbox.population(n=100)
result, log = algorithms.eaSimple(
    population, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, verbose=True
)
# 5. Get best individual
best_ind = tools.selBest(result, k=1)[0]
print("Best individual (weights):", best_ind)
print("Accuracy:", fitness(best_ind)[0])
print("miou:", fitness_mIoU(best_ind)[0])


gen	nevals
0  	100   
1  	63    
2  	55    
3  	62    
4  	63    
5  	57    
6  	64    
7  	55    
8  	72    
9  	64    
10 	67    
11 	44    
12 	66    
13 	64    
14 	66    
15 	60    


In [ ]:

weighted_scores = M

# Get the predicted class for each sample/pixel
predictions = np.argmax(weighted_scores, axis=1)

unique_classes = kitty_conf['learning_map']
iou_per_class = []

for cls in unique_classes:
	# True Positives: Pixels correctly predicted as class `cls`
	tp = np.sum((predictions == cls) & (L == cls))

	# False Positives: Pixels predicted as class `cls` but are actually other classes
	fp = np.sum((predictions == cls) & (L != cls))

	# False Negatives: Pixels that are class `cls` but predicted as other classes
	fn = np.sum((predictions != cls) & (L == cls))

	# Calculate IoU for the current class
	denominator = tp + fp + fn
	if denominator == 0:
		iou = 1.0 # If there are no true positives, false positives, or false negatives for this class,
					# and it's not present in ground truth or predictions, IoU is 1.0.
					# This case needs careful consideration. If the class is not present in L at all,
					# it's often ignored. If it's present but not predicted/true, then 0.
					# For simplicity, if the denominator is 0, it means the class was
					# neither present in ground truth nor predicted, so IoU is 1.
					# However, a common practice is to only calculate for classes present in ground truth.
	else:
		iou = tp / denominator
	iou_per_class.append(iou)

if not iou_per_class:
	print(0.0,) # No classes found, mIoU is 0

mIoU = np.mean(iou_per_class)
print("No mutation accuracy", np.mean(np.argmax((M),axis=1) == L))
print("No mutation miou", mIoU)

No mutation accuracy 0.6440669137069079
No mutation miou 0.8094430504102429
